In [1]:
import openpyxl
import os


# чтение файла построчно
def read_file_by_line(sheet, mincell, maxcell):
    lst = list()
    for rows in sheet[mincell:maxcell]:
        lst_in = list()
        for cell in rows:
            # значение ячейки
            cell_val = cell.value

            # замена None на 0
            if cell_val == None:
                cell_val = 0
            
            # проверка корректности данных
            # в ячейках с человеко-часами
            if str(cell_val).isdigit:
                lst_in.append(cell_val)
            else:
                print('Некорректные данные!')
            
        lst.append(lst_in)
    
    return lst


# значения ячеек по столбцам
def column_values(lst):
    lst_column = list()
    for i in range(len(lst[0])):
        lst_column_in = list()
        for j in range(len(lst)):
            lst_column_in.append(lst[j][i])
        lst_column.append(lst_column_in)
    
    return lst_column


# объединение значений по два столбца
def values_to_couple_PlanFact(lst):
    lst_couple = list()
    for i in range(0,len(lst),2):
        lst_couple.append(lst[i:i+2])
    
    return lst_couple


# получение списка успешностей сотрудников
def difference(lst):

    # success_workers_lst - список, содержащий успешность 
    # каждого сотрудника в порядке их записи в файле
    success_workers_lst = list()

    # элементы списка difference_lst - разница между фактом
    # и планом для каждого проекта для одного из сотрудников
    difference_lst = list()

    for i in range(len(lst)):
        for j in range(len(lst[0][0])):
            difference_lst.append(float(lst[i][1][j])-float(lst[i][0][j]))

        # присоединение разности к списку
        success_workers_lst.append(sum(difference_lst))

        # очистка списка перед тем, как приступить к обработке
        # успешности следующего сотрудника
        difference_lst.clear()
    
    return success_workers_lst


# получение имён сотрудников
def name_workers(sheet, max_col):
    workers = list()
    for col in range(5,max_col+1,2):
        workers.append((sheet.cell(row=1, column=col).value)[:-5])

    return workers


# успешность каждого сотрудника поимённо
def success_dict(workers,success_workers_lst):
    success_workers_dict = dict()
    for i in range(len(workers)):
        success_workers_dict[workers[i]] = success_workers_lst[i]
    
    return success_workers_dict



# обработка одного файла
def success_by_one_file(file_name):

    file = openpyxl.load_workbook(f'{file_name}')

    # выбор первого листа файла
    sheet = file.active

    # определение размера листа
    max_row = sheet.max_row
    max_col = sheet.max_column

    # определение координат начала и конца нужных данных 
    # (предполагается, что структура файлов та же, что в примере)
    mincell = 'E2' # по умолчанию
    maxcell = sheet.cell(row = max_row, column=max_col).coordinate

    # чтение файла построчно
    lst = read_file_by_line(sheet, mincell, maxcell)

    # значения ячеек по столбцам
    lst_columns = column_values(lst)

    # объединение значений по два столбца
    lst_couple = values_to_couple_PlanFact(lst_columns)

    # получение списка успешностей сотрудников
    success_workers_lst = difference(lst_couple)

    # получение имён сотрудников
    workers = name_workers(sheet, max_col)

    # успешность каждого сотрудника поимённо
    success_workers_dict_one_file = success_dict(workers,success_workers_lst)

    return success_workers_dict_one_file



# совмещение данных из файлов
def combination_dicts(lst_dicts):

    # содержит фио сотрудника и его успешность
    success_workers_dict_all_files = dict()

    # множество позволит не обрабатывать одних 
    # и тех же сотрудников несколько раз
    unic_workers = set()

    # перебор словарей
    for i in range(len(lst_dicts)):

        # key_i - рассматриваемый сотрудник
        for key_i in lst_dicts[i].keys():
            # values_lst - список успешностей для одного сотрудника из разных файлов
            values_lst = [lst_dicts[i].get(key_i)]

            # остальные словари
            for j in range(i+1, len(lst_dicts)):

                # если сотрудник упоминается в другом файле
                if key_i in lst_dicts[j].keys():
                    unic_workers.add(key_i)
                    values_lst.append(lst_dicts[j].get(key_i))
                # к итоговому словарю добавляем пару фио сотрудника - его успешность
                success_workers_dict_all_files[key_i] = sum(values_lst) 

            # если сотрудник есть только в одном файле,
            # его успешность добавляется без изменений
            if key_i not in unic_workers:
                success_workers_dict_all_files[key_i] = lst_dicts[i].get(key_i)
    
    return success_workers_dict_all_files


# обобщение на все файлы директории
def success_all_files():

    lst_dicts = list()
    # чтение всех файлов из директории и запись сотрудников и их
    # успешностей по отдельным словарям в список lst_dicts
    for elem in os.listdir():
        if elem[-4:] == 'xlsx':
            lst_dicts.append(success_by_one_file(elem))

    success_workers_dict_all_files = combination_dicts(lst_dicts)

    # печать сотрудников по успешности
    for elem in sorted(success_workers_dict_all_files, 
            key=success_workers_dict_all_files.get, reverse=True):
        print(elem)

 
success_all_files()

ValueError: could not convert string to float: 'aa'